In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
dataset = load_dataset("imdb")

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

%matplotlib inline

## Generating training set

In [4]:
text_train = pd.Series(dataset['train']['text'])
text_train[0:5]

0    I rented I AM CURIOUS-YELLOW from my video sto...
1    "I Am Curious: Yellow" is a risible and preten...
2    If only to avoid making this type of film in t...
3    This film was probably inspired by Godard's Ma...
4    Oh, brother...after hearing about this ridicul...
dtype: object

In [5]:
label_train = pd.Series(dataset['train']['label'])
label_train[0:5]

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [6]:
label_train.value_counts()

0    12500
1    12500
dtype: int64

In [7]:
# Directory for Colab (optional, use if running Colab)
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [8]:
import os
os.chdir('/content/gdrive/MyDrive/NLP_project')

In [9]:
!pwd

/content/gdrive/MyDrive/NLP_project


In [10]:
def create_train_data(dataset, seed=1803, n_train_samples=8000):
  ''' As default, we create 3 training sets of size 8000 for quantitative analysis. 
  Input:
  list_seed : list of seeds used
  n_train_samples : length of training sets
  '''
  # Import data
  text_train = pd.Series(dataset['train']['text'])
  label_train = pd.Series(dataset['train']['label'])
  n = len(text_train)

  np.random.seed(seed)
  indices = np.random.permutation(n)
  indices_samples = np.random.choice(indices, size=n_train_samples)
  text_train_samples = text_train[indices_samples]
  label_train_samples = label_train[indices_samples]
  df_samples = pd.DataFrame({'text': text_train_samples, 'label':label_train_samples})
  file_name = "pickle/imdb_train" +".pickle"
  df_samples.to_pickle(file_name)
  print(file_name + ' succesfully created')
  return None

In [11]:
create_train_data(dataset)

pickle/imdb_train.pickle succesfully created


In [12]:
df_train = pd.read_pickle("pickle/imdb_train.pickle")
df_train

,text,label
8305,"With one of my very favorite actors, James Spa...",0
10408,"$25,000 Pyramid Clues: Deep Blue Sea. Tremors....",0
19465,"If you're researching UFO facts, then this vid...",1
8775,This film concerns purportedly non-establishme...,0
16013,Phantom Lady (1944) Dir: Robert Siodmak <br />...,1
...,...,...
6271,"Ironically, although he can do the ""splits"", T...",0
21834,Bo Derek's beauty and John Derek's revolutiona...,1
1115,"OK, so it was written in 1996, before 9/11, so...",0
8107,"Did I miss something here? This ""adaptation"" h...",0


In [13]:
df_train.label.value_counts()

0    4007
1    3993
Name: label, dtype: int64

## Generating test set for BERT and textfooler

In [14]:
df = pd.read_csv("data/imdb/bert/textfooler/bert-base-uncased-imdb_textfooler.csv",sep=',')
df.head()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,[[Once]] again Mr. Costner has dragged out a [...,[[Whenever]] again Mr. Costner has dragged out...,0.000241,0.815027,0,1,0,469,Successful
1,[[This]] is an example of why the majority of ...,[[These]] is an example of why the majority of...,0.000183,0.554409,0,1,0,670,Successful
2,"First of all I hate those [[moronic]] rappers,...",First of all I hate those [[senseless]] rapper...,0.000289,0.716852,0,1,0,643,Successful
3,Not even the Beatles could write songs everyon...,Not even the Beatles could write songs everyon...,0.000303,0.960098,0,1,0,441,Successful
4,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.000311,0.952665,0,1,0,301,Successful


In [15]:
n_test = len(df)
n_test

10000

In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(df['ground_truth_output'], df['original_output'])

0.9306

In [17]:
df['result_type'].value_counts()

Successful    9175
Skipped        694
Failed         131
Name: result_type, dtype: int64

In [18]:
def clean_text(text):
  text = text.replace("[[", "")
  text = text.replace("]]", "")
  return text

In [19]:
df['original_text'] = df['original_text'].map(clean_text)
df['perturbed_text'] = df['perturbed_text'].map(clean_text)
df.head()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,Once again Mr. Costner has dragged out a movie...,Whenever again Mr. Costner has dragged out a s...,0.000241,0.815027,0,1,0,469,Successful
1,This is an example of why the majority of acti...,These is an example of why the majority of act...,0.000183,0.554409,0,1,0,670,Successful
2,"First of all I hate those moronic rappers, who...","First of all I hate those senseless rappers, w...",0.000289,0.716852,0,1,0,643,Successful
3,Not even the Beatles could write songs everyon...,Not even the Beatles could write songs everyon...,0.000303,0.960098,0,1,0,441,Successful
4,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.000311,0.952665,0,1,0,301,Successful


In [20]:
# Scenario 1

np.random.seed(1803)
indices = np.random.permutation(n_test)
n_max_adv = 2000
indices_adv = indices[:n_max_adv]
indices_clean = indices[n_max_adv:]
df_adv = df.loc[indices_adv,:]

mask_1 = df_adv['result_type']=='Successful' # Select only successful attacks
mask_2 = df_adv['original_output'] == df_adv['ground_truth_output'] # Select only correctly classified examples
df_adv = df_adv[mask_1 & mask_2]
n_adv = len(df_adv) # Number of attack samples

indices_clean_samples = np.random.choice(indices_clean, n_adv)
df_clean = df.loc[indices_clean_samples,:]

In [21]:
df_adv

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
3098,"So, I'm wondering while watching this film, di...","So, I'm wondering while watching this film, di...",0.000217,0.893735,0,1,0,333,Successful
288,"Although not a big Coen brothers fan, I am an ...","Although not a big Coen brothers fan, me am an...",0.000253,0.763794,0,1,0,1197,Successful
1686,I am in a movie club at my school and I was fo...,me am in a theatres nightclubs at my pupil and...,0.000198,0.533332,0,1,0,1595,Successful
5350,The Three Stooges has always been some of the ...,The Thirdly Stooges enjoys always been some of...,0.001343,0.624068,1,0,1,1560,Successful
8615,"(Possible ?? spoilers included, but nothing cr...","(Possible ?? spoilers included, but nothing cr...",0.001571,0.734415,1,0,1,207,Successful
...,...,...,...,...,...,...,...,...,...
5962,This movie gives Daniel Wu his chance to do a ...,This movie gives Daniel Wu his chance to do a ...,0.000268,0.509187,1,0,1,275,Successful
1859,I feel terribly sorry! Where the Lubitsch-pic ...,I feel terribly sorry! Then the Lubitsch-pic w...,0.000270,0.715104,0,1,0,306,Successful
9157,My curiosity and patience to finally see this ...,My curiosity and patience to finally see this ...,0.000272,0.960268,1,0,1,792,Successful
5007,I felt this film did have many good qualities....,I felt this film did have many good qualities....,0.000489,0.526733,1,0,1,160,Successful


In [22]:
df_clean

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
4555,"Well, it wasn't a complete waste. Armand was a...","Well, it wasn't a complete waste. Armand was a...",0.006082,0.533910,0,1,0,169,Successful
481,"When I think of a 1970s-period film, this is n...",Whenever I consider of a 1970s-period filmmake...,0.000163,0.679286,0,1,0,1499,Successful
770,This film is bad. Not so bad it is good. Just ...,This film is adverse. Not so bad it is good. M...,0.000279,0.526681,0,1,0,235,Successful
504,I knew five minutes after the monster made his...,I knew five minutes after the monsters made hi...,0.000220,0.555773,0,1,0,290,Successful
4608,"Wow, this movie is bad. Think ""Flashdance"" wit...","Wow, this scene is nasty. Think ""Flashdance"" w...",0.000270,0.569740,0,1,0,458,Successful
...,...,...,...,...,...,...,...,...,...
87,This would probably be a good film to see....p...,This would probably got a best film to see.......,0.000167,0.710496,0,1,0,450,Successful
4274,"I haven't seen a lot of episodes of ""Family Gu...","I haven't seen a lot of episodes of ""Family Gu...",0.000374,0.524431,0,1,0,338,Successful
1051,The film was half over before I managed to fig...,The cinema was half over before I managed to f...,0.006887,0.604653,0,1,0,148,Successful
6107,"When I first saw the movie, I thought it was s...","When I first saw the movie, I thought it was s...",0.000287,0.945250,1,0,1,179,Successful


In [23]:
adv_text = df_adv['perturbed_text']
adv_bool = np.ones(len(adv_text), dtype=int)
clean_text = df_clean['original_text']
clean_bool = np.zeros(len(clean_text), dtype=int)

text_test = pd.concat([adv_text, clean_text])
bool_test = np.concatenate([adv_bool, clean_bool])
df_test = pd.DataFrame({'text': text_test, 'adversarial': bool_test})

df_test

,text,adversarial
3098,"So, I'm wondering while watching this film, di...",1
288,"Although not a big Coen brothers fan, me am an...",1
1686,me am in a theatres nightclubs at my pupil and...,1
5350,The Thirdly Stooges enjoys always been some of...,1
8615,"(Possible ?? spoilers included, but nothing cr...",1
...,...,...
87,This would probably be a good film to see....p...,0
4274,"I haven't seen a lot of episodes of ""Family Gu...",0
1051,The film was half over before I managed to fig...,0
6107,"When I first saw the movie, I thought it was s...",0
